<a href="https://colab.research.google.com/github/caiodavic/Processamento-de-Linguagem-Natural-2022.1/blob/main/Named_Entity_Recognition_utilizando_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition utilizando Bidirectional LSTM
- Reconhecimento de Entidade Nomeada (NER), é uma subtarefa de extração de informações que busca localizar e classificar entidades nomeadas mencionadas em texto não estruturado em categorias predefinidas, como por exemplo: nomes de pessoas, organizações, locais, expressões de tempo, quantidades, valores monetários, entre outros.
- O dataset que vamos utilizar está disponível nesse [link do kaggle](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?select=ner_dataset.csv)

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/2022.1/PLN/ner_dataset.csv',encoding = 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


### Mapeamento palavras e tags para indexes
- Precisamos mapear cada palavra para cada índice do vocabulário.
- Também precisamos mapear cada tag para um índice

In [ ]:
def get_dict_map(data,token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
        idx2tok = {idx+1: tok for idx, tok in enumerate(vocab)}
        tok2idx = {tok: idx+1 for idx, tok in enumerate(vocab)}
    else:
        vocab = list(set(data['Tag'].to_list()))    
        idx2tok = {idx: tok for idx, tok in enumerate(vocab)}
        tok2idx = {tok: idx for idx, tok in enumerate(vocab)}

    return tok2idx,idx2tok

In [ ]:
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [ ]:
token2idx

{'blueprints': 1,
 'Top-ranked': 2,
 '163': 3,
 'renounce': 4,
 'denuded': 5,
 'Cirque': 6,
 'pilot-less': 7,
 'ricocheted': 8,
 'joyful': 9,
 'WTO': 10,
 'rider': 11,
 'Cual': 12,
 'rebate': 13,
 'Disease': 14,
 'Moroccan': 15,
 'Satan': 16,
 'recently-halted': 17,
 'Quake': 18,
 'Alamodome': 19,
 'Hoffman': 20,
 '325-member': 21,
 'offshore': 22,
 '11-year': 23,
 '8,599': 24,
 'comment': 25,
 'sub-contractor': 26,
 'Chrysanthemum': 27,
 'processed': 28,
 'sufficient': 29,
 'Troedsson': 30,
 'slowing': 31,
 'Hatim': 32,
 'three': 33,
 'birth': 34,
 'unifying': 35,
 'extensive': 36,
 'Retirement': 37,
 'judo': 38,
 'Let': 39,
 'reopens': 40,
 'ratio': 41,
 'cacao': 42,
 '\x97': 43,
 'locations': 44,
 'confiscate': 45,
 'snow': 46,
 'inject': 47,
 'favourite': 48,
 'bridged': 49,
 'MILOSEVIC': 50,
 "'ll": 51,
 'Laith': 52,
 'criminal': 53,
 'most-successful': 54,
 'Cumple': 55,
 'windy': 56,
 '0.7': 57,
 'veils': 58,
 'mediation': 59,
 'stabilizing': 60,
 'Gillespie': 61,
 'rocketed': 6

In [ ]:
idx2tag

{0: 'I-nat',
 1: 'I-per',
 2: 'I-org',
 3: 'I-tim',
 4: 'B-nat',
 5: 'B-geo',
 6: 'O',
 7: 'B-eve',
 8: 'I-geo',
 9: 'I-eve',
 10: 'I-gpe',
 11: 'B-tim',
 12: 'B-art',
 13: 'I-art',
 14: 'B-gpe',
 15: 'B-org',
 16: 'B-per'}

- Associando cada índice a cada palavra e tag no dataset

In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.sample(5)

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
4016,NaN,Simple,NNP,I-art,34681,13
373286,NaN,.,.,O,1218,6
110696,NaN,on,IN,O,32891,6
823095,NaN,dump,VB,O,34408,6
440660,NaN,address,NN,O,16644,6


In [ ]:
data_fillna = data.fillna(method='ffill', axis = 0)
data_fillna

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,13667,6
1,Sentence: 1,of,IN,O,27611,6
2,Sentence: 1,demonstrators,NNS,O,14240,6
3,Sentence: 1,have,VBP,O,15187,6
4,Sentence: 1,marched,VBN,O,15295,6
...,...,...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O,12933,6
1048571,Sentence: 47959,responded,VBD,O,11340,6
1048572,Sentence: 47959,to,TO,O,13782,6
1048573,Sentence: 47959,the,DT,O,24921,6


- Vamos agrupar os tokens por sentença e unir em listas.

In [ ]:
data_group = data_fillna.groupby(['Sentence #'], as_index=False)['Word','POS','Tag','Word_idx','Tag_idx'].agg(lambda x: list(x))
data_group.head()

<ipython-input-10-d15ce953158e>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['Sentence #'], as_index=False)['Word','POS','Tag','Word_idx','Tag_idx'].agg(lambda x: list(x))


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[13667, 27611, 14240, 15187, 15295, 6021, 4792...","[6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 5, 6, 6, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[20629, 250, 13628, 12933, 10932, 13782, 12488...","[14, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[20917, 19776, 34984, 2291, 10793, 32170, 2852...","[6, 6, 11, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 15..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[27967, 24892, 22868, 22900, 19802, 23581, 274...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[14346, 22544, 32489, 9452, 22616, 26840, 1967...","[5, 6, 6, 16, 1, 6, 11, 6, 5, 6, 14, 6, 14, 6,..."


### Preparação dos dados
- Precisamos transformar as sentenças em sequencias com os índices que temos em token2idx e fazer o padding para todas as sentenças terem o mesmo tamanho. 
- Também vamos transformar a tag de cada palavra em um vetor categorical.

In [ ]:
n_token = len(token2idx)

tokens = data_group.Word_idx.tolist()
maxlen = max([len(s) for s in tokens])
pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32',padding='post', value= 0)

In [ ]:
n_tags = len(tag2idx)

tags = data_group.Tag_idx.tolist()
pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32',padding='post', value= tag2idx["O"])
pad_tags_categorical = [to_categorical(i,num_classes=n_tags) for i in pad_tags]

In [ ]:
train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags_categorical, test_size=0.3, train_size=0.7, random_state=2022)

In [ ]:
print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tags length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
    )

train_tokens length: 33571 
train_tags length: 33571 
test_tokens length: 14388 
test_tags: 14388


### Construção do modelo
- Vamos construir uma rede neural recorrente com 4 camadas
  - Primeira camada é a camada de Embedding que já conhecemos anteriormente, ela receberá sequências de tamanho 104 e para cada token ela irá construir um vetor de dimensão 64.
  - A segunda camada é uma LSTM bidirecional, é uma camada recorrente. Ela obtém a saída da camada anterior e devido ao parâmetro ```merge_mode='concat'``` vai ser passada a informação dos dois fluxos da BiLSTM para a próxima camada. Nessa camada temos um dropout de 0.2.
  - A terceira camada é uma LSTM, com dropout de 0.5. 
  - Essa camada é reponsável por pegar a saída da camada LSTM com dimensão (104,256) e gera uma saída com dimensão 104 e cada item dessa dimensão é um vetor de 17 itens.







In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [ ]:
input_dim = n_token+1
output_dim = 64
input_length = maxlen

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = input_dim,output_dim=output_dim,input_length=input_length))
model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True,dropout=0.2,recurrent_dropout=0.2),merge_mode='concat'))
model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
model.add(TimeDistributed(Dense(n_tags, activation="relu")))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 64)           2251456   
                                                                 
 bidirectional_4 (Bidirectio  (None, 104, 128)         66048     
 nal)                                                            
                                                                 
 lstm_9 (LSTM)               (None, 104, 64)           49408     
                                                                 
 time_distributed_4 (TimeDis  (None, 104, 17)          1105      
 tributed)                                                       
                                                                 
Total params: 2,368,017
Trainable params: 2,368,017
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = model.fit(train_tokens,np.array(train_tags),batch_size=1024 ,epochs=5, validation_split = 0.2)

Epoch 1/5
27/27 [==============================] - 44s 1s/step - loss: 0.9847 - accuracy: 0.9301 - val_loss: 0.3115 - val_accuracy: 0.9679
Epoch 2/5
27/27 [==============================] - 37s 1s/step - loss: 0.2790 - accuracy: 0.9678 - val_loss: 0.2223 - val_accuracy: 0.9679
Epoch 3/5
27/27 [==============================] - 36s 1s/step - loss: 0.2585 - accuracy: 0.9678 - val_loss: 0.1990 - val_accuracy: 0.9679
Epoch 4/5
27/27 [==============================] - 38s 1s/step - loss: 0.2098 - accuracy: 0.9678 - val_loss: 0.1874 - val_accuracy: 0.9679
Epoch 5/5
27/27 [==============================] - 36s 1s/step - loss: 0.1886 - accuracy: 0.9678 - val_loss: 0.1742 - val_accuracy: 0.9679


In [ ]:
predicts = model.predict(test_tokens)

450/450 [==============================] - 38s 83ms/step


In [ ]:
def get_sentence_in_text(idx2token,sentence,idx2tag,predict):
    sentence_complete = []
    for i,j in zip(sentence,predict):
        if i != 0:  
            word = idx2token[i]
            tag = idx2tag[np.argmax(j)]
            sentence_complete.append((word,tag))
    
    return ' '.join(i[0] if i[1] == 'O' else f'{i[0]}[{i[1]}]' for i in sentence_complete)   

In [ ]:
sentence_complete = get_sentence_in_text(idx2token,test_tokens[60],idx2tag,test_tags[60])
sentence_complete

'Former Peruvian[B-gpe] President[B-per] Alberto[I-per] Fujimori[I-per] has announced he will run for president again in the upcoming election .'